# PROJEKT2

In [20]:
import requests
import pandas as pd
from bs4 import BeautifulSoup
import re
from urllib.parse import urljoin
import json
import matplotlib.pyplot as plt
import numpy as np
from sumy.parsers.plaintext import PlaintextParser
from sumy.nlp.tokenizers import Tokenizer
from sumy.summarizers.lsa import LsaSummarizer
import sqlite3
from sqlite_dump import iterdump

### ADRESY FIRM

In [21]:
def get_location(company_name):
    base_url = "https://nominatim.openstreetmap.org/search"
    headers = {
            "User-Agent": "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_11_5) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/50.0.2661.102 Safari/537.36"
        }
    params = {
            "q": company_name,
            "format": "json",
            "addressdetails": 1  
        }
    response = requests.get(base_url, params=params, headers = headers)
    data = response.json()
    if data:
        location = data[0]["display_name"]
        return location
    else:
        return None

### PODSUMOWANIE OFERTY

In [22]:
def create_summary(doc):
    clean = re.compile('<.*?>') #aby uniknac bledow z tagami html
    clean_text = re.sub(clean, '', str(doc))  
    clean_text = clean_text.replace('"', '-') #aby uniknac bledow
    
    parser = PlaintextParser.from_string(clean_text, Tokenizer("english"))
    summarizer = LsaSummarizer()
    
    summary = summarizer(parser.document, 2)
    summary_text = ' '.join([str(sentence) for sentence in summary])

    
    return summary_text

### ZAKTUALIZOWANE OFERTY PRACY (O PODSUMOWANIE I ADRES)

Niestety strona pracuj.pl uległa zmianie i nie znajduje już ofert, jednak pozostawiłam to bez zmian, bo drugi projekt tego nie wymagał, a miałam dane ze strony justjoinit

In [23]:
def pracuj():
    results = []
    existing_records = set()
    
    start = 1
    while True:
        url = f"https://it.pracuj.pl/praca/krakow;wp?rd=0&et=17%2C4%2C18&sal=1&pn={start}&its=big-data-science"
        response = requests.get(url, verify=True)
        if response.status_code != 200:
            raise Exception('Failed to load page {}'.format(url))
        doc = BeautifulSoup(response.text, 'html.parser')
        offers = doc.select('.c1s2myew .c1wygkax .core_n194fgoq')
        pages = [urljoin(url, link['href']) for link in offers if 'https://www.pracuj.pl/praca/' in link['href']]
        for page in pages:
            response = requests.get(page)
            if response.status_code != 200:
                raise Exception('Failed to load page {}'.format(page))
            doc = BeautifulSoup(response.text, 'html.parser')
            zrodlo = 'pracuj.pl'
            link = page
            stanowisko = doc.select('.offer-viewkHIhn3')[0].text
            firma = doc.select('.offer-viewwtdXJ4')[0].text
            minimalne = doc.select('.offer-viewiafL8R .offer-viewZGJhIB, .offer-viewYo2KTr')[0].text
            minimalne = minimalne.replace(',', '.')
            minimalne = re.sub(r'[^\d.]', '', minimalne)
            maksymalne = doc.select('.offer-viewYo2KTr')[0].text
            maksymalne = maksymalne.replace(',', '.')
            maksymalne = re.sub(r'[^\d.]', '', maksymalne)

            stawka = doc.select('.offer-viewSGW6Yi')[0].text

            if minimalne:
                if 'hr' in stawka or 'godz' in stawka:
                    if 'net' in stawka or 'netto' in stawka:
                        minimalne = float(minimalne)
                        minimalne *= 1.23 * 168
                        minimalne = int(minimalne)
                    else:
                        minimalne *=168
                else:
                    if 'net' in stawka or 'netto' in stawka:
                        minimalne = float(minimalne)
                        minimalne *= 1.23
                        minimalne = float(minimalne)
                    else:
                        minimalne = int(float(minimalne)) 
            else:
                minimalne = None
                
            if maksymalne:
                if 'hr' in stawka or 'godz' in stawka:
                    if 'net' in stawka or 'netto' in stawka:
                        maksymalne = float(maksymalne)
                        maksymalne *= 1.23 * 168
                        maksymalne = int(maksymalne)
                    else:
                        maksymalne *=168
                else:
                    if 'net' in stawka or 'netto' in stawka:
                        maksymalne = float(maksymalne)
                        maksymalne *= 1.23
                        maksymalne = float(maksymalne)
                    else:
                        maksymalne = int(float(maksymalne)) 
            else:
                maksymalne = None


            waluta = "PLN"
            umiejetnosci = [umiejetnosc.text for umiejetnosc in doc.select('.offer-viewfjH4z3:first-of-type .offer-viewU0gxPf')]
            
            kategoria = doc.select('.offer-viewPFKc0t')[0].text #CZY TO RECZNIE WPISAC?
            seniority = doc.select('.offer-viewXo2dpV')[2].text
            seniorities = [seniority.strip() for seniority in seniority.split(',')]

            seniority_mapping = {
                'specialist (Mid / Regular)': 'Mid',
                'senior specialist (Senior)': 'Senior',
                'specialist (Mid / Regular)': 'Mid',
                'specjalista (Mid / Regular)': 'Mid',
                'starszy specjalista (Senior)': 'Senior',
                'senior specialist': 'Senior',
                'junior specialist (Junior)': 'Junior',
                'mÅodszy specjalista (Junior)': 'Junior',
                'specialist': 'Mid'
            }

            standardized_seniorities = [seniority_mapping.get(seniority, seniority) for seniority in seniorities if seniority in seniority_mapping]
            priority_order = ['Junior', 'Mid', 'Senior']

            sorted_seniorities = sorted(standardized_seniorities, key=lambda x: priority_order.index(x))

            seniority = ', '.join(sorted_seniorities)
            if ',' in seniority:
                second_seniority = seniority.split(',')[1].strip()
            else:
                second_seniority = seniority.strip()
                
            record_key = (zrodlo, link, stanowisko, firma, minimalne, maksymalne, waluta, tuple(umiejetnosci), kategoria, second_seniority)
            if record_key not in existing_records:
                existing_records.add(record_key)
                results.append({
                    'Źródło': zrodlo,
                    'Link': link,
                    'Stanowisko': stanowisko,
                    'Firma': firma,
                    'Minimalne_wynagrodzenie': minimalne,
                    'Maksymalne_wynagrodzenie': maksymalne,
                    'Waluta': waluta,
                    'Umiejetnosci': umiejetnosci,
                    'Kategoria': kategoria,
                    'Seniority': second_seniority,
                    'Adres' : get_location(firma),
                    'Podsumowanie' : create_summary(doc)
                })

            
        start += 1
        if not pages:
            break
    return results

In [5]:
def justjoinit():
    results = []
    url = "https://justjoin.it/krakow/data/experience-level_junior.mid.senior/with-salary_yes"
    response = requests.get(url)
    if response.status_code != 200:
        raise Exception('Failed to load page {}'.format(url))

    doc = BeautifulSoup(response.text, 'html.parser')
    offers = doc.select('.offer_list_offer_link.css-4lqp8g')
    pages = [urljoin(url, link['href']) for link in offers]
    for page in pages:
        response = requests.get(page)
        if response.status_code != 200:
            raise Exception('Failed to load page {}'.format(url))
            continue
        doc = BeautifulSoup(response.text, 'html.parser')
        zrodlo = 'justjoin.it'
        link = page
        stanowisko = doc.select('.css-vb54bv .css-1u65tlp')[0].text
        firma = doc.select('.css-mbkv7r')[0].text
        minimalne = doc.select('.css-1pavfqb')[0].text.split('-',1)[0]
        maksymalne  = doc.select('.css-1pavfqb')[0].text.split('-',1)[1].rstrip(' PLN')
        minimalne = minimalne.replace(" ", "")   
        maksymalne = maksymalne.replace(" ", "")   

        waluta = doc.select('.css-1pavfqb')[0].text.split(' ', 6)[5]
        umiejetnosci = [umiejetnosc.text for umiejetnosc in doc.select('h6.css-x1xnx3')]
        seniority = doc.select('.css-15wyzmd')[1].text
        kategoria = doc.select('.css-6t6cyr')[0].text

        results.append({
            'Źródło': zrodlo,
            'Link': link,
            'Stanowisko': stanowisko ,
            'Firma': firma,
            'Minimalne_wynagrodzenie' : minimalne ,
            'Maksymalne_wynagrodzenie' : maksymalne,
            'Waluta' : waluta,
            'Umiejetnosci' : umiejetnosci ,
            'Kategoria' : kategoria,
            'Seniority' : seniority,
            'Adres' : get_location(firma),
            'Podsumowanie' : create_summary(doc)
        })
    return results

In [24]:
wyniki_pracuj = pracuj()  
wyniki_justjoin = justjoinit()

wszystkie_wyniki = wyniki_pracuj + wyniki_justjoin 

In [27]:
wyniki_df = pd.DataFrame(wszystkie_wyniki)
wyniki_df.head(5)

,Źródło,Link,Stanowisko,Firma,Minimalne_wynagrodzenie,Maksymalne_wynagrodzenie,Waluta,Umiejetnosci,Kategoria,Seniority,Adres,Podsumowanie
0,justjoin.it,https://justjoin.it/offers/limango-polska-expe...,Experienced Machine Learning Engineer,Limango Polska,16000,19000,PLN,"[Machine Learning, PySpark, Google Cloud, SQL,...",Data,Mid,None,If you're looking for a company that shares th...
1,justjoin.it,https://justjoin.it/offers/datumo-senior-data-...,Senior Data Engineer,Datumo,20000,30000,PLN,"[Cloud, Big Data, Airflow, Spark, Snowflake, P...",Data,Senior,None,The primary task involves implementing and ext...
2,justjoin.it,https://justjoin.it/offers/pragmile-data-engin...,Data Engineer,Pragmile,23520,26880,PLN,[Data],Data,Senior,None,Odmowa podania danych obowiązkowych może skutk...
3,justjoin.it,https://justjoin.it/offers/itds-machine-learni...,Machine Learning Engineer,ITDS,18000,26000,PLN,"[Python, Machine Learning, GCP, Data Science, ...",Data,Senior,"Amvox / APPR / Asperion / ITDS, Gooimeer, Naar...",Internal users from the Regulatory Compliance ...
4,justjoin.it,https://justjoin.it/offers/remitly-data-engineer,Data Engineer,Remitly,20500,23000,PLN,"[AWS, Data, RDBMS, Python, Scala, Big Data, Do...",Data,Mid,None,"Since 2011, we have been tirelessly delivering..."


In [26]:
output_file = 'all_data.json'
with open(output_file, 'w', encoding='utf-8') as file:
    json.dump(wszystkie_wyniki, file, ensure_ascii=False, indent=4)  

# CZ.2

In [28]:
with open('all_data.json', encoding= 'utf-8') as f:
    wyniki = json.load(f)

In [29]:
def create_database():
    conn = sqlite3.connect('abcd.db')
    cursor = conn.cursor()
    
    
    cursor.execute('''
    CREATE TABLE IF NOT EXISTS Stanowisko (
        ID INTEGER PRIMARY KEY,
        Nazwa TEXT
    )
    ''')

    cursor.execute('''
    CREATE TABLE IF NOT EXISTS Firma (
        ID INTEGER PRIMARY KEY,
        Nazwa TEXT,
        Adres TEXT
    )
    ''')

    cursor.execute('''
    CREATE TABLE IF NOT EXISTS Umiejętność (
        ID INTEGER PRIMARY KEY,
        Nazwa TEXT
    )
    ''')

    cursor.execute('''
    CREATE TABLE IF NOT EXISTS Waluta (
        ID INTEGER PRIMARY KEY,
        Nazwa TEXT
    )
    ''')

    cursor.execute('''
    CREATE TABLE IF NOT EXISTS Źródło (
        ID INTEGER PRIMARY KEY,
        Nazwa TEXT
    )
    ''')

    cursor.execute('''
    CREATE TABLE IF NOT EXISTS Kategoria (
        ID INTEGER PRIMARY KEY,
        Nazwa TEXT
    )
    ''')
    
    cursor.execute('''
    CREATE TABLE IF NOT EXISTS Oferta (
        ID INTEGER PRIMARY KEY,
        ID_Stanowiska INTEGER,
        ID_Firmy INTEGER,
        ID_Kategorii INTEGER,
        ID_Waluty INTEGER,
        ID_Źródła INTEGER,
        Link TEXT,
        Umiejętności TEXT,
        Seniority TEXT,
        Wynagrodzenie_MIN INTEGER,
        Wynagrodzenie_MAX INTEGER,
        Podsumowanie TEXT,
        FOREIGN KEY (ID_Stanowiska) REFERENCES Stanowisko(ID),
        FOREIGN KEY (ID_Firmy) REFERENCES Firma(ID),
        FOREIGN KEY (ID_Kategorii) REFERENCES Kategoria(ID),
        FOREIGN KEY (ID_Waluty) REFERENCES Waluta(ID),
        FOREIGN KEY (ID_Źródła) REFERENCES Źródło(ID)
    )
    ''')


    conn.commit()
    conn.close()

In [30]:
create_database()

In [31]:
def insert_data(df):
    conn = sqlite3.connect('abcd.db', timeout=10)
    cursor = conn.cursor()

    #źródło, stanowisko, waluta, kategoria 
    kolumny = ['Źródło', 'Stanowisko', 'Waluta', 'Kategoria']
    
    try:
        conn.execute('BEGIN TRANSACTION')
        for index, row in df.iterrows():
            for kol in kolumny:
                wart = row[kol]
                cursor.execute(f'''
                    INSERT INTO {kol} (Nazwa)
                    SELECT '{wart}'
                    WHERE NOT EXISTS (SELECT 1 FROM {kol} WHERE Nazwa = '{wart}')
                ''')
    #firma 
            firma = row['Firma']
            adres = row['Adres']
            if adres is not None:
                adres = adres.replace("'", '|')
            cursor.execute(f'''
                    INSERT INTO Firma (Nazwa, Adres) SELECT "{firma}", '{adres}'
                   WHERE NOT EXISTS (SELECT 1 from Firma where Nazwa = "{firma}"  AND Adres = "{adres}")
                ''')    
            
    #umiejetnosci
            umiej = row['Umiejetnosci']
            umiej_ids = []
            for u in umiej:
                cursor.execute(f'''
                        INSERT INTO Umiejętność (Nazwa) SELECT "{u}"
                       WHERE NOT EXISTS (SELECT 1 from Umiejętność where Nazwa = "{u}")
                    ''')    
                cursor.execute(f'''SELECT ID from Umiejętność where Nazwa = "{u}"
                ''')
                umiej_id = cursor.fetchone()[0]
                umiej_ids.append(umiej_id)
                
    #oferta 
            cursor.execute(f'''SELECT ID FROM Stanowisko WHERE Nazwa = '{row['Stanowisko']}'
            ''')
            id_stanowisko = cursor.fetchone()[0]

            cursor.execute(f'''SELECT ID FROM Firma WHERE Nazwa = '{firma}' AND Adres = '{adres}'
            ''')
            id_firma = cursor.fetchone()[0]

            cursor.execute(f'''SELECT ID FROM Kategoria WHERE Nazwa = '{row['Kategoria']}'
            ''')
            id_kategoria = cursor.fetchone()[0]

            cursor.execute(f'''SELECT ID FROM Waluta WHERE Nazwa = '{row['Waluta']}'
            ''')
            id_waluta = cursor.fetchone()[0]

            cursor.execute(f'''SELECT ID FROM Źródło WHERE Nazwa = '{row['Źródło']}'
            ''')
            id_zrodlo = cursor.fetchone()[0]
            
            link = row['Link']
            umiej_ids_str = ','.join(map(str, umiej_ids))

            senior = row['Seniority']
            wyn_min = row['Minimalne_wynagrodzenie']
            wyn_max = row['Maksymalne_wynagrodzenie']
            podsum = row['Podsumowanie']

            cursor.execute(f'''
            INSERT INTO Oferta (ID_Stanowiska, ID_Firmy, ID_Kategorii, ID_Waluty, ID_Źródła, Link, Umiejętności, Seniority, 
                    Wynagrodzenie_MIN, Wynagrodzenie_MAX, Podsumowanie) SELECT {id_stanowisko}, {id_firma},
                    {id_kategoria}, {id_waluta}, {id_zrodlo}, "{link}", "{umiej_ids_str}", "{senior}", {wyn_min},
                    {wyn_max} , "{podsum}"
                    ''')         
        
    
        conn.commit()
    except sqlite3.Error as e:
        conn.rollback()
        print(f"An error occurred: {e}")
    finally:
        conn.close()


In [32]:
insert_data(wyniki_df)

In [33]:
conn = sqlite3.connect('abcd.db')

with open('abcd_dump.sql', 'w', encoding='utf-8') as f:
    for line in iterdump(conn):
        f.write(f'{line}\n')

conn.close()